In [1]:
import os
import FinanceDataReader as fdr
from tqdm import tqdm
import pandas as pd

# Pycaret 이용

## 모든 종목 예측

* 종가와 상관계수 높은 특성 : KOSDAQ(0.87), KOSPI(0.79), DOW(0.58), NASDAQ(0.50), S&P500(0.56)
* 예측 순서(종가와 상관계수 높은 특성 먼저 예측)
    * weekday, weeknumdm로 KOSDAQ 예측(종가와 상관계수 0.87)
    * weekday, weeknum, KOSDAQ으로 NASDAQ 예측(KOSDAQ과 상관계수 0.68)
    * weekday, weeknum, KOSDAQ, NASDAQ으로 S&P500 예측(NASDAQ과 상관계수 0.93)
    * weekday, weeknum, KOSDAQ, NASDAQ, S&P500으로 DOW 예측(S&P500과 상관계수 0.95)
    * weekday, weeknum, KOSDAQ, NASDAQ, S&P500, DOW로 KOSPI 예측
    * weekday, weeknum, KOSDAQ, NASDAQ, S&P500, DOW, KOSPI로 다른 종가 예측 

In [2]:
# 제출 점수 :

In [3]:
from pycaret.regression import *

# 데이터 불러오기

## 외부 데이터 가져오기

In [4]:
# 외부 데이터 가져오기
df = pd.read_csv('../data/20210104_20211119_data_interpolate.csv')
df.head()

,Date,exchange_rate,kospi,kosdaq,kospi100,dow,nasdaq,sp500,미국채10년-2년,BTC,Gold,Oil,US10Y,US2Y
0,2021-01-04,1086.48,2944.45,977.62,3052.74,30223.89,12698.4,3700.65,0.8009,36499000.0,37.10,32.38,0.918,0.1171
1,2021-01-05,1086.42,2990.57,985.76,3099.42,30391.60,12819.0,3726.86,0.8320,37962000.0,37.18,33.94,0.955,0.1230
2,2021-01-06,1087.93,2968.21,981.39,3070.05,30829.40,12740.8,3748.14,0.9002,40731000.0,36.60,34.11,1.039,0.1388
3,2021-01-07,1094.28,3031.68,988.86,3139.48,31041.13,13067.5,3803.79,0.9402,44575000.0,36.48,34.53,1.081,0.1408
4,2021-01-08,1092.93,3152.18,987.79,3293.96,31097.97,13202.0,3824.68,0.9821,47241000.0,35.26,35.43,1.119,0.1369


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230 entries, 0 to 229
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           230 non-null    object 
 1   exchange_rate  230 non-null    float64
 2   kospi          230 non-null    float64
 3   kosdaq         230 non-null    float64
 4   kospi100       230 non-null    float64
 5   dow            230 non-null    float64
 6   nasdaq         230 non-null    float64
 7   sp500          230 non-null    float64
 8   미국채10년-2년      230 non-null    float64
 9   BTC            230 non-null    float64
 10  Gold           230 non-null    float64
 11  Oil            230 non-null    float64
 12  US10Y          230 non-null    float64
 13  US2Y           230 non-null    float64
dtypes: float64(13), object(1)
memory usage: 25.3+ KB


In [6]:
df.columns

Index(['Date', 'exchange_rate', 'kospi', 'kosdaq', 'kospi100', 'dow', 'nasdaq',
       'sp500', '미국채10년-2년', 'BTC', 'Gold', 'Oil', 'US10Y', 'US2Y'],
      dtype='object')

### 필요한 특성만 추출

In [7]:
df = df[['Date','kosdaq','nasdaq','sp500','dow','kospi']]
df.head()

,Date,kosdaq,nasdaq,sp500,dow,kospi
0,2021-01-04,977.62,12698.4,3700.65,30223.89,2944.45
1,2021-01-05,985.76,12819.0,3726.86,30391.60,2990.57
2,2021-01-06,981.39,12740.8,3748.14,30829.40,2968.21
3,2021-01-07,988.86,13067.5,3803.79,31041.13,3031.68
4,2021-01-08,987.79,13202.0,3824.68,31097.97,3152.18


In [9]:
# 요일 번호와 주 번호 지정하여 열로 추가하기
df['Date'] = pd.to_datetime(df['Date'])
df['weekday'] = df['Date'].apply(lambda x : x.weekday())
df['weeknum'] = df['Date'].apply(lambda x : x.strftime('%V'))

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230 entries, 0 to 229
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Date     230 non-null    datetime64[ns]
 1   kosdaq   230 non-null    float64       
 2   nasdaq   230 non-null    float64       
 3   sp500    230 non-null    float64       
 4   dow      230 non-null    float64       
 5   kospi    230 non-null    float64       
 6   weekday  230 non-null    int64         
 7   weeknum  230 non-null    object        
dtypes: datetime64[ns](1), float64(5), int64(1), object(1)
memory usage: 14.5+ KB


In [11]:
df.columns

Index(['Date', 'kosdaq', 'nasdaq', 'sp500', 'dow', 'kospi', 'weekday',
       'weeknum'],
      dtype='object')

In [12]:
df = df[['Date', 'weekday', 'weeknum','kosdaq', 'nasdaq', 'sp500', 'dow', 'kospi']]
df.head()

,Date,weekday,weeknum,kosdaq,nasdaq,sp500,dow,kospi
0,2021-01-04,0,01,977.62,12698.4,3700.65,30223.89,2944.45
1,2021-01-05,1,01,985.76,12819.0,3726.86,30391.60,2990.57
2,2021-01-06,2,01,981.39,12740.8,3748.14,30829.40,2968.21
3,2021-01-07,3,01,988.86,13067.5,3803.79,31041.13,3031.68
4,2021-01-08,4,01,987.79,13202.0,3824.68,31097.97,3152.18


### 훈련, 테스트 셋 나누기
* 훈련셋 기간 : 2021-10-25 ~ 2021-11-19
* 테스트셋 기간 : 2021-11-29 ~ 2021-12-3

In [13]:
start_date = '2021-10-25'
end_date = '2021-11-19'

In [14]:
train = df[(df['Date'] <= end_date) & (df['Date'] >= start_date)]
train = train.reset_index()
train = train.drop('index',axis=1)

In [15]:
train.shape

(20, 8)

In [16]:
test = pd.DataFrame(pd.date_range('2021-11-29','2021-12-3',freq='B'), columns = ['Date'])
test['Date'] = pd.to_datetime(test['Date'])
test['weekday'] = test['Date'].apply(lambda x : x.weekday())
test['weeknum'] = test['Date'].apply(lambda x : x.strftime('%V'))
test

,Date,weekday,weeknum
0,2021-11-29,0,48
1,2021-11-30,1,48
2,2021-12-01,2,48
3,2021-12-02,3,48
4,2021-12-03,4,48


In [17]:
train.head()

,Date,weekday,weeknum,kosdaq,nasdaq,sp500,dow,kospi
0,2021-10-25,0,43,994.31,15226.7,4566.48,35743.78,3020.54
1,2021-10-26,1,43,1011.76,15235.7,4574.79,35755.83,3049.08
2,2021-10-27,2,43,1008.95,15235.8,4551.68,35491.48,3025.49
3,2021-10-28,3,43,1000.13,15448.1,4596.42,35729.89,3009.55
4,2021-10-29,4,43,992.33,15498.4,4605.38,35819.59,2970.68


In [18]:
train.tail()

,Date,weekday,weeknum,kosdaq,nasdaq,sp500,dow,kospi
15,2021-11-15,0,46,1029.03,15853.8,4682.81,36087.98,2999.52
16,2021-11-16,1,46,1035.46,15973.9,4700.90,36144.13,2997.21
17,2021-11-17,2,46,1031.26,15921.6,4688.67,35931.52,2962.42
18,2021-11-18,3,46,1032.77,15993.7,4706.64,35871.34,2947.38
19,2021-11-19,4,46,1041.92,16057.4,4697.96,35602.18,2971.02


In [19]:
test.head()

,Date,weekday,weeknum
0,2021-11-29,0,48
1,2021-11-30,1,48
2,2021-12-01,2,48
3,2021-12-02,3,48
4,2021-12-03,4,48


## 종목 정보 가져오기

In [20]:
# 종목 정보 가져오기
path = '../data'
list_name = 'Stock_List.csv'
stock_list = pd.read_csv(os.path.join(path,list_name))
stock_list.head()

,종목명,종목코드,상장시장
0,삼성전자,5930,KOSPI
1,SK하이닉스,660,KOSPI
2,NAVER,35420,KOSPI
3,카카오,35720,KOSPI
4,삼성바이오로직스,207940,KOSPI


In [21]:
stock_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 370 entries, 0 to 369
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   종목명     370 non-null    object
 1   종목코드    370 non-null    int64 
 2   상장시장    370 non-null    object
dtypes: int64(1), object(2)
memory usage: 8.8+ KB


In [22]:
# 종목 코드 6자리로 맞추기
stock_list['종목코드'] = stock_list["종목코드"].astype(str).str.zfill(6)
stock_list.head()

,종목명,종목코드,상장시장
0,삼성전자,005930,KOSPI
1,SK하이닉스,000660,KOSPI
2,NAVER,035420,KOSPI
3,카카오,035720,KOSPI
4,삼성바이오로직스,207940,KOSPI


In [23]:
stock_list.to_csv('stock_list.csv', index=False)

In [24]:
stock_list = pd.read_csv('stock_list.csv')
stock_list.head()

,종목명,종목코드,상장시장
0,삼성전자,5930,KOSPI
1,SK하이닉스,660,KOSPI
2,NAVER,35420,KOSPI
3,카카오,35720,KOSPI
4,삼성바이오로직스,207940,KOSPI


# 종가 예측

* 1. kosdaq 예측
* 2. nasdaq
* 3. sp500
* 4. dow
* 5. kospi
* 6. 종가

### 테스트셋의 5가지 특성 예측 하기

In [25]:
features = ['Date', 'weekday', 'weeknum','kosdaq', 'nasdaq', 'sp500', 'dow', 'kospi']
for i in range(5):
    model = setup(train, target = features[i+3], ignore_features = features[i+4:], fold=4, fold_shuffle=True, silent=True)
    tuned_model = tune_model(compare_models(sort='MAPE', n_select=1))
    final_model = finalize_model(tuned_model)
    end_features = features[:i+3]
    pred = predict_model(final_model, test)
    test[features[i+3]] = pred['Label']

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,3.7925,23.5985,4.8578,-0.0426,0.0016,0.0013
1,45.6322,2143.4846,46.2978,-0.3022,0.0156,0.0155
2,23.2374,865.2014,29.4143,-0.1093,0.0098,0.0077
3,20.2885,670.1422,25.8871,-1.5922,0.0087,0.0068
Mean,23.2377,925.6067,26.6143,-0.5116,0.0089,0.0078
SD,14.9024,769.0508,14.7407,0.6311,0.0050,0.0051


In [26]:
test

,Date,weekday,weeknum,kosdaq,nasdaq,sp500,dow,kospi
0,2021-11-29,0,48,994.300008,15516.016764,4613.382896,35954.460938,2971.095891
1,2021-11-30,1,48,1009.960002,15730.499891,4652.258954,36125.671875,3003.252271
2,2021-12-01,2,48,997.817456,15516.016764,4613.382896,35941.957031,2971.845081
3,2021-12-02,3,48,997.817456,15516.016764,4613.382896,35830.054688,2977.736577
4,2021-12-03,4,48,1000.700012,15516.016764,4613.382896,35812.945312,2977.736577


In [27]:
test.to_csv('20211025_1119_pred_1129_1203_test.csv', index=False)

In [28]:
test = pd.read_csv('20211025_1119_pred_1129_1203_test.csv')
test

,Date,weekday,weeknum,kosdaq,nasdaq,sp500,dow,kospi
0,2021-11-29,0,48,994.300008,15516.016764,4613.382896,35954.460,2971.095891
1,2021-11-30,1,48,1009.960002,15730.499891,4652.258954,36125.670,3003.252271
2,2021-12-01,2,48,997.817456,15516.016764,4613.382896,35941.957,2971.845081
3,2021-12-02,3,48,997.817456,15516.016764,4613.382896,35830.055,2977.736577
4,2021-12-03,4,48,1000.700012,15516.016764,4613.382896,35812.945,2977.736577


In [29]:
train.to_csv('20211025_1119_train.csv', index=False)

In [30]:
train = pd.read_csv('20211025_1119_train.csv')
train.head()

,Date,weekday,weeknum,kosdaq,nasdaq,sp500,dow,kospi
0,2021-10-25,0,43,994.31,15226.7,4566.48,35743.78,3020.54
1,2021-10-26,1,43,1011.76,15235.7,4574.79,35755.83,3049.08
2,2021-10-27,2,43,1008.95,15235.8,4551.68,35491.48,3025.49
3,2021-10-28,3,43,1000.13,15448.1,4596.42,35729.89,3009.55
4,2021-10-29,4,43,992.33,15498.4,4605.38,35819.59,2970.68


In [32]:
train.tail()

,Date,weekday,weeknum,kosdaq,nasdaq,sp500,dow,kospi
15,2021-11-15,0,46,1029.03,15853.8,4682.81,36087.98,2999.52
16,2021-11-16,1,46,1035.46,15973.9,4700.90,36144.13,2997.21
17,2021-11-17,2,46,1031.26,15921.6,4688.67,35931.52,2962.42
18,2021-11-18,3,46,1032.77,15993.7,4706.64,35871.34,2947.38
19,2021-11-19,4,46,1041.92,16057.4,4697.96,35602.18,2971.02


In [31]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Date     20 non-null     object 
 1   weekday  20 non-null     int64  
 2   weeknum  20 non-null     int64  
 3   kosdaq   20 non-null     float64
 4   nasdaq   20 non-null     float64
 5   sp500    20 non-null     float64
 6   dow      20 non-null     float64
 7   kospi    20 non-null     float64
dtypes: float64(5), int64(2), object(1)
memory usage: 1.4+ KB


In [54]:
stock_list['종목코드'].values[0]

'005930'

In [57]:
code_data = fdr.DataReader('005930', start = start_date, end = end_date)[['Close']].reset_index()
train['Close'] = code_data['Close']
train

,Date,weekday,weeknum,kosdaq,nasdaq,sp500,dow,kospi,Close
0,2021-10-04,0,40,969.285,14255.5,4300.46,34003.58,2990.675,72200.0
1,2021-10-05,1,40,955.370,14433.8,4345.72,34315.99,2962.170,71300.0
2,2021-10-06,2,40,922.360,14501.9,4363.55,34417.98,2908.310,71600.0
3,2021-10-07,3,40,953.430,14654.0,4399.76,34754.15,2959.460,71500.0
4,2021-10-08,4,40,953.110,14579.5,4391.36,34746.71,2956.300,69000.0
5,2021-10-11,0,41,946.630,14486.2,4361.19,34496.85,2936.340,68800.0
6,2021-10-12,1,41,940.150,14465.9,4350.64,34378.08,2916.380,69400.0
7,2021-10-13,2,41,953.470,14571.6,4363.80,34378.60,2944.410,70100.0
8,2021-10-14,3,41,983.430,14823.4,4438.23,34911.70,2988.640,70200.0
9,2021-10-15,4,41,990.540,14897.3,4471.37,35295.48,3015.060,70600.0
